# Implement GLM and Contrast Function
A generalized linear model is used to find where brain activity changes given different tasks
The contrast function compares the brain activity between target and general task.

We are interested in the following contrast:
- Differentiate brain region activity between target and general task during audio condition
- Differentiate brain region activity between target and general task during visual condition
- Differentiate brain region activity between audio and visual condition after contrast is applied to both conditions

In [6]:
# import dependencies
from nilearn.glm.first_level import FirstLevelModel
import numpy as np

In [7]:
# Implement glm base model
def fit_glm(fmri_data, events_data, tr):
    """ 
    Fit a glm to the fmri data
    """
    model = FirstLevelModel(t_r = tr)
    glm = model.fit(fmri_data, events = events_data)
    return glm

In [8]:
# Categorize stimuli into target and general task
def categorize_stimuli(events_data, file_path):
    """ 
    Categorize stimuli based on the filename patterns
    For visual data: 'inh' = general task, 'sel' = target task
    For audio data: NA
    events_data: pandas dataframe pulled from the events.tsv file
    file_path: path to the file
    """
    if 'visual' in file_path: # if there is a data file with visual in the path
        if 'inh' in file_path:
            events_data['condition'] = 'general' # inhibition task
        elif 'sel' in file_path:
            events_data['condition'] = 'target' # selection task

    #TODO: Need to add audio data categorization
    
    return events_data

In [9]:
# Implement contrast function
def contrast_glm_stimuli(glm, events_data, file_path, contrast_def = {'target': 1, 'general': -1}): 
    """ 
    Apply a contrast to the glm results 
    """
    events = categorize_stimuli(events_data, file_path)
    contrast = glm.compute_contrast(contrast_def)
    return contrast

def contrast_glm_conditions(glm, contrast_def = {'audio': 1, 'visual': -1}):
    """ 
    Apply a contrast to the glm results
    """
    contrast = glm.compute_contrast(contrast_def)
    return contrast

In [15]:
import os
import glob
import pandas as pd

# Run the glm and contrast
def run_glm_contrast(fmri_data, data_path, tr, file_path):
    """ 
    Run the glm on the fmri data and apply contrast
    """
    # get all events files in the events_data_path
    pattern = os.path.join(data_path, 'sub-*/func/*events.tsv')
    events_files = glob.glob(pattern, recursive = True)

    if not events_files:
        raise ValueError(f"No events files found in {pattern}")
    
    # read the first events file 
    events_data = pd.read_csv(events_files[0], sep = '\t')

    # fit the glm
    glm = fit_glm(fmri_data, events_data, tr)

    # apply contrast
    contrast_stimuli = contrast_glm_stimuli(glm, events_data, file_path)
    contrast_conditions = contrast_glm_conditions(glm)
    return contrast_stimuli, contrast_conditions

data_path = "../data/visual"
file_path = "../results/visual/cleaned_fmri_data.nii.gz" 
fmri_data = "../results/visual/cleaned_fmri_data.nii.gz" #TODO: need to add audio data
tr = 1.5 # test with visual data
run_glm_contrast(fmri_data, data_path, tr, file_path)

ValueError: File not found: '../results/visual/cleaned_fmri_data.nii.gz'